In [1]:
# Standard libraries
import os
import json
import re
from datetime import datetime
from datetime import datetime, timedelta

from uuid import uuid4

# Third-party libraries
import requests
import pinecone
from bs4 import BeautifulSoup
from tqdm import tqdm
from tqdm.auto import tqdm

# LangChain libraries
import langchain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate

import matplotlib.pyplot as plt
import seaborn as sns



c:\Users\takat\.virtualenvs\Diploma-ESySJfXn\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo-1106',
    temperature=0.0
)
total_days = 364
interval = 7
num_dates = total_days // interval

# Generating the dates with 5-day intervals
start_date = datetime(2023, 1, 2)
new_dates = [start_date + timedelta(days=i * interval) for i in range(num_dates)]
new_dates_formatted = [date.strftime('%Y-%m-%d') for date in new_dates]

# new_dates_formatted, len(new_dates_formatted)

In [3]:
new_dates_formatted

['2023-01-02',
 '2023-01-09',
 '2023-01-16',
 '2023-01-23',
 '2023-01-30',
 '2023-02-06',
 '2023-02-13',
 '2023-02-20',
 '2023-02-27',
 '2023-03-06',
 '2023-03-13',
 '2023-03-20',
 '2023-03-27',
 '2023-04-03',
 '2023-04-10',
 '2023-04-17',
 '2023-04-24',
 '2023-05-01',
 '2023-05-08',
 '2023-05-15',
 '2023-05-22',
 '2023-05-29',
 '2023-06-05',
 '2023-06-12',
 '2023-06-19',
 '2023-06-26',
 '2023-07-03',
 '2023-07-10',
 '2023-07-17',
 '2023-07-24',
 '2023-07-31',
 '2023-08-07',
 '2023-08-14',
 '2023-08-21',
 '2023-08-28',
 '2023-09-04',
 '2023-09-11',
 '2023-09-18',
 '2023-09-25',
 '2023-10-02',
 '2023-10-09',
 '2023-10-16',
 '2023-10-23',
 '2023-10-30',
 '2023-11-06',
 '2023-11-13',
 '2023-11-20',
 '2023-11-27',
 '2023-12-04',
 '2023-12-11',
 '2023-12-18',
 '2023-12-25']

In [4]:
qa = ChatPromptTemplate.from_template(
'''
System command: You are ChatGPT a Large Language Model trained by OpenAI to chat with humans. Todays date is {date}.
Your knowledge cutoff is 2023-01-10.
Your task: {task}
'''
)
chain = qa | llm
results = []
lengths_running = [0] * len(new_dates_formatted)
all_results  = {}
tasks = [
    "Write an essay on the French Revolution.",
    "Write an essay on the strongest jaws in the animal kingdom.",
    "Write an essay on the history of the United States.",
    "Write an essay on option pricing.",
    
]
for task in tasks:
    results = []
    print(task)
    for date in new_dates_formatted:
        print(date)
        result = chain.invoke({'date':date, 'task':task}).content
        # print(result)
        results.append(result)

    
    lengths = [len(s) for s in results]
    lengths_running = [a + b for a, b in zip(lengths, lengths_running)]
    all_results[task] = results

# Example results with strings of varying lengths
# results = ["example", "some text", "short", "a longer example string", "text", "another", "one more", "string", "data", "sample", "yet another", "final"]

# Calculating the length of each string in results
# lengths = [len(s) for s in results]

# Plotting the lengths against dates
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")
sns.lineplot(x=new_dates_formatted, y=lengths_running, )
plt.xticks(rotation=45)
plt.xlabel("Date")
plt.ylabel("Length of String")
plt.title("String Lengths Over Time")
plt.tight_layout()
plt.show()


Write an essay on the French Revolution.
2023-01-02
2023-01-09
2023-01-16
2023-01-23
2023-01-30
2023-02-06
2023-02-13
2023-02-20
2023-02-27
2023-03-06
2023-03-13
2023-03-20
2023-03-27
2023-04-03
2023-04-10
2023-04-17
2023-04-24
2023-05-01
2023-05-08
2023-05-15
2023-05-22
2023-05-29
2023-06-05
2023-06-12
2023-06-19
2023-06-26
2023-07-03
2023-07-10
2023-07-17
2023-07-24
2023-07-31
2023-08-07
2023-08-14
2023-08-21
2023-08-28
2023-09-04
2023-09-11
2023-09-18
2023-09-25
2023-10-02
2023-10-09
2023-10-16
2023-10-23
2023-10-30
2023-11-06
2023-11-13
2023-11-20
2023-11-27
2023-12-04
2023-12-11
2023-12-18
2023-12-25
Write an essay on the strongest jaws in the animal kingdom.
2023-01-02
2023-01-09
2023-01-16
2023-01-23
2023-01-30
2023-02-06
2023-02-13
2023-02-20
2023-02-27
2023-03-06
2023-03-13
2023-03-20
2023-03-27
2023-04-03
2023-04-10
2023-04-17
2023-04-24
2023-05-01
2023-05-08
2023-05-15
2023-05-22
2023-05-29
2023-06-05
2023-06-12
2023-06-19
2023-06-26
2023-07-03
2023-07-10
2023-07-17
2023-07-

In [ ]:
# Save all_results as a JSON file
file_path = './documents/unrelated/yearly-mondays.json'
with open(file_path, 'w') as file:
    json.dump(all_results, file)

file_path

'./documents/unrelated/yearly.json'